**Machine Learning Techniques**


1.   Linear Regression
2.   Random Forest Regressor
3.   SVM(Support Vector Machine)
4.   SVR with different kernels






In [1]:
import pandas as pd
import numpy as np
df1=pd.read_excel('//content//drive//MyDrive//energy.xlsx')

In [2]:
df1.head(), df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37966 entries, 0 to 37965
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DateTime                37966 non-null  datetime64[ns]
 1   HolidayFlag             37966 non-null  int64         
 2   DayOfWeek               37966 non-null  int64         
 3   WeekOfYear              37966 non-null  int64         
 4   Day                     37966 non-null  int64         
 5   Month                   37966 non-null  int64         
 6   Year                    37966 non-null  int64         
 7   PeriodOfDay             37966 non-null  int64         
 8   ForecastWindProduction  37966 non-null  object        
 9   SystemLoadEA            37966 non-null  object        
 10  SMPEA                   37966 non-null  object        
 11  ORKTemperature          37966 non-null  object        
 12  ORKWindspeed            37966 non-null  object

(             DateTime  HolidayFlag  DayOfWeek  WeekOfYear  Day  Month  Year  \
 0 2021-11-01 00:00:00            0          1          44    1     11  2021   
 1 2021-11-01 00:30:00            0          1          44    1     11  2021   
 2 2021-11-01 01:00:00            0          1          44    1     11  2021   
 3 2021-11-01 01:30:00            0          1          44    1     11  2021   
 4 2021-11-01 02:00:00            0          1          44    1     11  2021   
 
    PeriodOfDay ForecastWindProduction SystemLoadEA  SMPEA ORKTemperature  \
 0            0                 315.31      3388.77  49.26           6.00   
 1            1                 321.80      3196.66  49.26           6.00   
 2            2                 328.57      3060.71  49.10           5.00   
 3            3                 335.60      2945.56  48.04           6.00   
 4            4                 342.90      2849.34  33.75           6.00   
 
   ORKWindspeed CO2Intensity ActualWindProduction Syst

**DATA PREPROCESSING**

In [3]:
# Convert object columns to numeric where applicable
numeric_columns = ['ForecastWindProduction', 'SystemLoadEA', 'SMPEA',
                   'ORKTemperature', 'ORKWindspeed', 'CO2Intensity',
                   'ActualWindProduction', 'SystemLoadEP2', 'SMPEP2']

# Replace commas (if any) and convert to numeric
for col in numeric_columns:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')

# Verify the conversions
print(df1.dtypes)

# Ensure no NaN values are present after conversion
if df1.isnull().sum().sum() > 0:
    print("Warning: Missing values introduced after type conversion.")
else:
    print("All columns successfully converted to numeric.")

DateTime                  datetime64[ns]
HolidayFlag                        int64
DayOfWeek                          int64
WeekOfYear                         int64
Day                                int64
Month                              int64
Year                               int64
PeriodOfDay                        int64
ForecastWindProduction           float64
SystemLoadEA                     float64
SMPEA                            float64
ORKTemperature                   float64
ORKWindspeed                     float64
CO2Intensity                     float64
ActualWindProduction             float64
SystemLoadEP2                    float64
SMPEP2                           float64
dtype: object


In [4]:
df1.isnull().sum()

,0
DateTime,0
HolidayFlag,0
DayOfWeek,0
WeekOfYear,0
Day,0
Month,0
Year,0
PeriodOfDay,0
ForecastWindProduction,5
SystemLoadEA,2


In [5]:
df1.head()

,DateTime,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
0,2021-11-01 00:00:00,0,1,44,1,11,2021,0,315.31,3388.77,49.26,6.0,9.3,600.71,356.0,3159.60,54.32
1,2021-11-01 00:30:00,0,1,44,1,11,2021,1,321.80,3196.66,49.26,6.0,11.1,605.42,317.0,2973.01,54.23
2,2021-11-01 01:00:00,0,1,44,1,11,2021,2,328.57,3060.71,49.10,5.0,11.1,589.97,311.0,2834.00,54.23
3,2021-11-01 01:30:00,0,1,44,1,11,2021,3,335.60,2945.56,48.04,6.0,9.3,585.94,313.0,2725.99,53.47
4,2021-11-01 02:00:00,0,1,44,1,11,2021,4,342.90,2849.34,33.75,6.0,11.1,571.52,346.0,2655.64,39.87


In [6]:
# Convert the 'DateTime' column to pandas datetime format if not already
df1['DateTime'] = pd.to_datetime(df1['DateTime'], errors='coerce')

# Extract useful components from the 'DateTime' column (e.g., year, month, day, etc.)
df1['Year'] = df1['DateTime'].dt.year
df1['Month'] = df1['DateTime'].dt.month
df1['Day'] = df1['DateTime'].dt.day
df1['DayOfWeek'] = df1['DateTime'].dt.dayofweek  # Monday=0, Sunday=6
df1['Hour'] = df1['DateTime'].dt.hour

# Drop the original 'DateTime' column if it's no longer needed
df1 = df1.drop(columns=['DateTime'])

# Verify the transformation
print(df1.head())


   HolidayFlag  DayOfWeek  WeekOfYear  Day  Month  Year  PeriodOfDay  \
0            0          0          44    1     11  2021            0   
1            0          0          44    1     11  2021            1   
2            0          0          44    1     11  2021            2   
3            0          0          44    1     11  2021            3   
4            0          0          44    1     11  2021            4   

   ForecastWindProduction  SystemLoadEA  SMPEA  ORKTemperature  ORKWindspeed  \
0                  315.31       3388.77  49.26             6.0           9.3   
1                  321.80       3196.66  49.26             6.0          11.1   
2                  328.57       3060.71  49.10             5.0          11.1   
3                  335.60       2945.56  48.04             6.0           9.3   
4                  342.90       2849.34  33.75             6.0          11.1   

   CO2Intensity  ActualWindProduction  SystemLoadEP2  SMPEP2  Hour  
0        600.71  

In [7]:
# Drop rows with missing values
df1 = df1.dropna()

# Check if all missing values are eliminated
print("Number of missing values after dropping rows:", df1.isnull().sum().sum())


Number of missing values after dropping rows: 0


In [8]:
df1.isnull().sum()

,0
HolidayFlag,0
DayOfWeek,0
WeekOfYear,0
Day,0
Month,0
Year,0
PeriodOfDay,0
ForecastWindProduction,0
SystemLoadEA,0
SMPEA,0


In [9]:
df1.head(), df1.shape

(   HolidayFlag  DayOfWeek  WeekOfYear  Day  Month  Year  PeriodOfDay  \
 0            0          0          44    1     11  2021            0   
 1            0          0          44    1     11  2021            1   
 2            0          0          44    1     11  2021            2   
 3            0          0          44    1     11  2021            3   
 4            0          0          44    1     11  2021            4   
 
    ForecastWindProduction  SystemLoadEA  SMPEA  ORKTemperature  ORKWindspeed  \
 0                  315.31       3388.77  49.26             6.0           9.3   
 1                  321.80       3196.66  49.26             6.0          11.1   
 2                  328.57       3060.71  49.10             5.0          11.1   
 3                  335.60       2945.56  48.04             6.0           9.3   
 4                  342.90       2849.34  33.75             6.0          11.1   
 
    CO2Intensity  ActualWindProduction  SystemLoadEP2  SMPEP2  Hour  
 0

Training model using Train Test Split

In [10]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X= df1.drop('SystemLoadEA', axis=1)
y = df1['SystemLoadEA']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=0)

**LINEAR REGRESSION**

In [ ]:
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print("Linear Regression")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Linear Regression
Mean Squared Error (MSE): 22012.37
Root Mean Squared Error (RMSE): 148.37
R-squared (R²): 0.97


**RANDOM FOREST REGRESSOR**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
y_rf=rf.predict(X_test)

mse = mean_squared_error(y_test, y_rf)
r2 = r2_score(y_test, y_rf)
rmse = np.sqrt(mse)

print("Random Forest Regressor")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Random Forest Regressor
Mean Squared Error (MSE): 11851.63
Root Mean Squared Error (RMSE): 108.87
R-squared (R²): 0.98


**SVM**

In [ ]:
from sklearn import svm
svm=svm.SVR()
svm.fit(X_train,y_train)
y_svm=svm.predict(X_test)

mse = mean_squared_error(y_test, y_svm)
r2 = r2_score(y_test, y_svm)
rmse = np.sqrt(mse)

print("SVM")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

SVM
Mean Squared Error (MSE): 32729.23
Root Mean Squared Error (RMSE): 180.91
R-squared (R²): 0.96


**SVR using RBF KERNEL**

In [ ]:
from sklearn import svm
rbf=svm.SVR(kernel='rbf')
rbf.fit(X_train,y_train)
y_rbf=rbf.predict(X_test)

mse = mean_squared_error(y_test, y_rbf)
r2 = r2_score(y_test, y_rbf)
rmse = np.sqrt(mse)

print("SVM")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

SVM
Mean Squared Error (MSE): 32729.23
Root Mean Squared Error (RMSE): 180.91
R-squared (R²): 0.96


**SVR using LINEAR KERNEL**

In [ ]:
from sklearn import svm
lin=svm.SVR(kernel='linear')
lin.fit(X_train,y_train)
y_lin=lin.predict(X_test)

mse = mean_squared_error(y_test, y_lin)
r2 = r2_score(y_test, y_lin)
rmse = np.sqrt(mse)

print("SVM with linear kernel")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

SVM with linear kernel
Mean Squared Error (MSE): 22193.39
R-squared (R²): 0.97


**SVR using SIGMOID KERNEL**

In [ ]:
from sklearn import svm
sig=svm.SVR(kernel="sigmoid")
sig.fit(X_train,y_train)
y_sig=sig.predict(X_test)

mse = mean_squared_error(y_test, y_sig)
r2 = r2_score(y_test, y_sig)
rmse = np.sqrt(mse)

print("SVM with sigmoid kernel")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

SVM with sigmoid kernel
Mean Squared Error (MSE): 934450.61
Root Mean Squared Error (RMSE): 966.67
R-squared (R²): -0.27


**SVR using QUADRATIC KERNEL**

In [ ]:
from sklearn import svm
quad=svm.SVR(kernel="poly", degree=2, gamma=0.001)
quad.fit(X_train,y_train)
y_quad=quad.predict(X_test)

mse = mean_squared_error(y_test, y_quad)
r2 = r2_score(y_test, y_quad)
rmse = np.sqrt(mse)

print("SVM with quadratic kernel")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

**SVR using CUBIC KERNEL**

In [ ]:
from sklearn import svm
cub=svm.SVR(kernel="poly", degree=3, gamma=0.0001,C=1)
cub.fit(X_train,y_train)
y_cub=cub.predict(X_test)

mse = mean_squared_error(y_test, y_cub)
r2 = r2_score(y_test, y_cub)
rmse = np.sqrt(mse)

print("SVM with cubic kernel")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

**Gradient Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor()
gb.fit(X_train,y_train)
y_gb=gb.predict(X_test)

mse = mean_squared_error(y_test, y_gb)
r2 = r2_score(y_test, y_gb)
rmse = np.sqrt(mse)

print("Gradient boosting")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")


Gradient boosting
Mean Squared Error (MSE): 17836.62
Root Mean Squared Error (RMSE): 133.55
R-squared (R²): 0.97


**Adaptive Boosting**

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor()
ada.fit(X_train,y_train)
y_ada=ada.predict(X_test)

mse = mean_squared_error(y_test, y_ada)
r2 = r2_score(y_test, y_ada)
rmse = np.sqrt(mse)

print("Adaptive boosting")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Adaptive boosting
Mean Squared Error (MSE): 34013.71
Root Mean Squared Error (RMSE): 184.43
R-squared (R²): 0.95


**Categorical Boosting**

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
cat=CatBoostRegressor()
cat.fit(X_train,y_train)
y_cat=cat.predict(X_test)

mse = mean_squared_error(y_test, y_cat)
r2 = r2_score(y_test, y_cat)
rmse = np.sqrt(mse)

print("Categorical boosting")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Learning rate set to 0.070115
0:	learn: 790.5635324	total: 56.6ms	remaining: 56.5s
1:	learn: 742.6630228	total: 64.5ms	remaining: 32.2s
2:	learn: 697.3766703	total: 74.8ms	remaining: 24.9s
3:	learn: 655.6914733	total: 82.8ms	remaining: 20.6s
4:	learn: 616.0462093	total: 89.6ms	remaining: 17.8s
5:	learn: 578.8908479	total: 96.7ms	remaining: 16s
6:	learn: 544.6268089	total: 104ms	remaining: 14.7s
7:	learn: 512.7106123	total: 110ms	remaining: 13.7s
8:	learn: 483.8544154	total: 118ms	remaining: 13s
9:	learn: 455.6743297	total: 124ms	remaining: 12.3s
10:	learn: 429.9313226	total: 132ms	remaining: 11.8s
11:	learn: 406.3303297	total: 138ms	remaining: 11.4s
12:	learn: 384.2559872	total: 145ms	remaining: 11s
13:	learn: 364.0625340	total: 152ms	remaining: 10.7s
14:	learn: 345.3583108	total: 159ms	remaining: 10.4s
15:	learn: 328.0706074	total: 166ms	remaining: 10.2s
16:	learn: 311.6955015	total: 173ms	remaining: 9.98s
17:	learn: 296.6953757	total: 179ms	remaining: 9.78s
18:	learn: 282.9567227	tot

Extreme Gradient Boosting

In [ ]:
from xgboost import XGBRegressor
xgb=XGBRegressor()
xgb.fit(X_train,y_train)
y_xgb=xgb.predict(X_test)

mse= mean_squared_error(y_test, y_xgb)
r2= r2_score(y_test, y_xgb)
rmse= np.sqrt(mse)

print('Extreme Gradient Boosting')
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Extreme Gradient Boosting
Mean Squared Error (MSE): 6701.41
Root Mean Squared Error (RMSE): 81.86
R-squared (R²): 0.99


In [ ]:
X=np.array([y_rf,y_xgb,y_cat]).T
meta_learner= RandomForestRegressor()
meta_learner.fit(X,y_test)
e_pred=meta_learner.predict(X)

mse= mean_squared_error(y_test, e_pred)
r2= r2_score(y_test, e_pred)
rmse= np.sqrt(mse)

print('Customized Ensemble Model')
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

Customized Ensemble Model
Mean Squared Error (MSE): 919.62
Root Mean Squared Error (RMSE): 30.33
R-squared (R²): 1.00


In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, Lasso
ridgeReg = Ridge(alpha=10)

ridgeReg.fit(X_train,y_train)

#train and test scorefor ridge regression
train_score_ridge = ridgeReg.score(X_train, y_train)
test_score_ridge = ridgeReg.score(X_test, y_test)

print("\nRidge Model\n")
print("The train score for ridge model is {}".format(train_score_ridge))
print("The test score for ridge model is {}".format(test_score_ridge))


Ridge Model

The train score for ridge model is 0.9689401049929534
The test score for ridge model is 0.9678191916242567


In [ ]:
from sklearn.linear_model import RidgeCV

#ridge Cross validation
ridge_cv = RidgeCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 10]).fit(X_train, y_train)

#score
print("The train score for ridge model is {}".format(ridge_cv.score(X_train, y_train)))
print("The test score for ridge model is {}".format(ridge_cv.score(X_test, y_test)))

The train score for ridge model is 0.9689401049919942
The test score for ridge model is 0.9678191905206307
